# Konlpy를 이용해서 고유명사, 일반명사, 형용사를 Question 문장 앞에 붙여주기

In [2]:
!pip install konlpy
!pip install datasets
!pip uninstall -y pyarrow  # 버전 안맞아서 삭제 후 재설치
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
Found existing installation: pyarrow 15.0.0
Uninstalling pyarrow-15.0.0:
  W

In [4]:
from konlpy.tag import Kkma
"""
    Summary:
    이 코드는 arrow파일의 데이터셋을 건드려서, question의 앞에 고유명사, 일반명사, 형용사를 한 번 더 붙여주는 코드입니다.
    새로운 파일로 생성되어서 저장되며, train과 test 데이터셋 모두에 적용합니다.
"""

def modify_sentence(sentence):
    kkma = Kkma()
    morphemes = kkma.pos(sentence)

    # 고유명사, 일반 명사, 형용사 원형만 선택
    selected_morphemes = [word for word, pos in morphemes if pos in ['NNG', 'NNP'] or pos.startswith('VA')]

    modified_sentence = ', '.join(selected_morphemes) + ', ' + sentence
    return modified_sentence

def modify_sentence_list(sentence_list):
    modified_list = [modify_sentence(sentence) for sentence in sentence_list]
    return modified_list

# 예시
original_sentence = "지구에서 가장 큰 새는?"
modified_sentence = modify_sentence(original_sentence)
print(modified_sentence)

지구, 크, 새, 지구에서 가장 큰 새는?


In [ ]:
from datasets import load_from_disk

# 데이터셋 로드
train_dataset_path = "./"  # 실제 데이터셋 경로로 수정
train_dataset = load_from_disk(train_dataset_path)

# question 열에 전처리 함수 적용
train_dataset = train_dataset.map(lambda example: {'question': modify_sentence_list(example['question'])}, batched=True)

# 변경된 데이터셋을 새로운 Arrow 파일로 저장
output_train_dataset_path = "./data/test_dataset_preprocessed"  # 새로운 디렉토리 경로로 수정
train_dataset.save_to_disk(output_train_dataset_path)

Map:   0%|          | 0/3952 [00:00<?, ? examples/s]

In [ ]:
# csv 파일로 보기
from datasets import load_from_disk
import pandas as pd

# 데이터셋 로드
train_dataset_path = "./data/test_dataset_preprocessed"  # 수정된 경로로 변경
train_dataset = load_from_disk(train_dataset_path)

# 데이터셋을 판다스 DataFrame으로 변환
df = pd.DataFrame(train_dataset)

# CSV 파일로 저장
csv_output_path = "./data/test_dataset_preprocessed.csv"  # 원하는 저장 경로로 변경
df.to_csv(csv_output_path, index=False)